In [ ]:
# %% 

""" Created on November 13, 2023 // @author: Sarah Shi """

import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

!pip install torch torchaudio torchvision torchtext
!pip install -i https://test.pypi.org/simple/ mineralML==0.0.0.0
import mineralML as mm

from google.colab import files

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


We have loaded in the mineralML Python package with trained machine learning models for classifying minerals. Examples workflows working with these spectra can be found on the [ReadTheDocs](https://mineralML.readthedocs.io/en/latest/). 

The Google Colab implementation here aims to get your electron microprobe compositions classified and processes. We remove degrees of freedom to simplify the process. The igneous minerals considered for this study include: amphibole, apatite, biotite, clinopyroxene, garnet, ilmenite, K-feldspar, magnetite, muscovite, olivine, orthopyroxene, plagioclase, quartz, rutile, spinel, tourmaline, and zircon. 

The files necessary include a CSV file containing your electron microprobe analyses in oxide weight percentages. Find an example [here](https://github.com/sarahshi/mineralML/blob/main/Validation_Data/lepr_allphases_lim.csv). The necessary oxides are $SiO_2$, $TiO_2$, $Al_2O_3$, $FeO_t$, $MnO$, $MgO$, $CaO$, $Na_2O$, $K_2O$, $Cr_2O_3$. For the oxides not analyzed for specific minerals, the preprocessing will fill in the nan values as 0. 

We will apply both supervised and unsupervised machine learning models to the dataset. 

# I. Supervised Machine Learning with Bayesian Neural Networks with Variational Inference

# Load your CSV file here: 

In [ ]:

data_directory = "/content/YOUR_DATA/"

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Change the current working directory
os.chdir(data_directory)

# Upload the files
uploaded_files = files.upload()


# Read in the CSV file and prepare for analysis (fill in nans, limit to trained igneous minerals): 

In [ ]:

# Read in your dataframe of mineral data, called DF.csv. 
# Prepare the dataframe by removing rows with too many NaNs, filling some with zeros, and filtering to the minerals described by mineralML. 

df_load = mm.load_df('DF.csv')
df_nn, _ = mm.prep_df_nn(df_load)


# Apply the Bayesian neural network with variational inference to your data:

In [ ]:

df_pred_nn, probability_matrix = mm.predict_class_prob_nn(df_nn)


# Examine the classifications report for your microanalyses, plot performance: 

In [ ]:

# Look at the predicted mineral dataframe. 

df_pred_nn


In [ ]:

# Create a classification report to determine the accuracy, precision, f1, etc. 

bayes_valid_report = classification_report(
    df_nn['Mineral'], df_pred_nn['Predict_Mineral'], zero_division=0
)
print("LEPR Validation Report:\n", bayes_valid_report)


In [ ]:

# Create and plot a confusion matrix 

cm = mm.confusion_matrix_df(df_nn['Mineral'], df_pred_nn['Predict_Mineral'])
print("LEPR Confusion Matrix:\n", cm)
cm[cm < len(df_pred_nn['Predict_Mineral'])*0.0005] = 0
mm.pp_matrix(cm, savefig = 'none') 


Excellent, these classifications now provide the most likely minerals, along with associated probabilities. Let's turn to unsupervised learning, to visualize these minerals in latent space. 

# I. Unsupervised Machine Learning with Autoencoders and Clustering (HDBSCAN)

# Prepare the same CSV as above for analysis: 

In [ ]:

df_ae, _ = mm.prep_df_ae(df_load)


# Apply the Bayesian neural network with variational inference to your data:

In [ ]:

df_pred_ae = mm.predict_class_prob_ae(df_ae)


# Examine the output: 

In [ ]:

df_pred_ae


# Plot your data in latent space: 

In [ ]:

mm.plot_latent_space(df_pred_ae)
